In [1]:
import requests
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
# API request to get product info for 1 specific product
url = "https://www.duluthtrading.com/api/ShopperProducts/getProducts?ids=76020&allImages=true&perPricebook=true"
response = requests.request("GET", url)
data = response.json()

In [3]:
# Data class to parse information from json/ dictionary tree
class TShirt:
    def __init__(self, product):
        self.id = product['productId']
        if 'name' in product:
            self.name = product['name']
        else:
            self.name = ""
        self.orderable = product['orderable']
        self.price = product['price']
        self.size = product['variationValues']['size']
        self.color = product['variationValues']['color']
        self.regprice = product['tieredPrices'][0]['price']
        
    def __str__(self):
        print()
        return f"""
                 Product ID: {self.id}
                 Name: {self.name}
                 Orderable: {self.orderable}
                 Price: {self.price}
                 Regular Price: {self.regprice}
                 Size: {self.size}
                 Color: {self.color}
        """
    
    def to_dict(self):
        return {
                 "Id": self.id,
                 "Name": self.name,
                 "Orderable": self.orderable,
                 "Price": self.price,
                 "RegPrice": self.regprice,
                 "Size": self.size,
                 "Color": self.color
            }
                          

In [4]:
# Loop through all product variants and add to list
tshirt_list = []
for product_type in data['data']:
    url = product_type['slugUrl']
    for product in product_type['variants']:
        try:
            tshirt = TShirt(product).to_dict()
            tshirt["link"] = url
            tshirt_list.append(tshirt)
        except:
            print(product)

In [5]:
# Convert from dictionary to dataframe
df = pd.DataFrame.from_dict(tshirt_list)

# Add column to show the sale percentage
df['SalePercent'] = (1 - df['Price'] / df['RegPrice'])

# Grab data to translate color coding
color_df = pd.DataFrame.from_dict(data['data'][0]['variationAttributes'][1]['values'])[['name','value']]

# Merge product table to color coding table & remove extra column
df = df.merge(color_df, left_on='Color', right_on='value')
df = df[['Id','Name','Price','Orderable', 'SalePercent','Size','Color','name','link']]
df = df.rename(columns = {'name':'ColorName'})
df.sort_values(by='SalePercent', ascending=False, na_position='last', inplace=True)

In [6]:
df

,Id,Name,Price,Orderable,SalePercent,Size,Color,ColorName,link
45,000220915,Men's Longtail T Relaxed Fit Short Sleeve T-Sh...,13.99,False,0.43,SM,BHR,Blue Harbor,https://www.duluthtrading.com/mens-longtail-t-...
36,000524137,Men's Longtail T Relaxed Fit Short Sleeve T-Sh...,14.99,True,0.39,SM,AMR,Amber Brown,https://www.duluthtrading.com/mens-longtail-t-...
37,000524138,Men's Longtail T Relaxed Fit Short Sleeve T-Sh...,14.99,True,0.39,XLG,AMR,Amber Brown,https://www.duluthtrading.com/mens-longtail-t-...
35,000524136,Men's Longtail T Relaxed Fit Short Sleeve T-Sh...,14.99,True,0.39,MED,AMR,Amber Brown,https://www.duluthtrading.com/mens-longtail-t-...
34,000524135,Men's Longtail T Relaxed Fit Short Sleeve T-Sh...,14.99,True,0.39,LRG,AMR,Amber Brown,https://www.duluthtrading.com/mens-longtail-t-...
...,...,...,...,...,...,...,...,...,...
27,000050126,Men's Longtail T Relaxed Fit Short Sleeve T-Sh...,24.50,True,0.00,SM,WHI,White,https://www.duluthtrading.com/mens-longtail-t-...
28,000079411,Men's Longtail T Relaxed Fit Short Sleeve T-Sh...,24.50,True,0.00,SM,DGY,Dark Gray,https://www.duluthtrading.com/mens-longtail-t-...
29,000079412,Men's Longtail T Relaxed Fit Short Sleeve T-Sh...,24.50,True,0.00,MED,DGY,Dark Gray,https://www.duluthtrading.com/mens-longtail-t-...
30,000079413,Men's Longtail T Relaxed Fit Short Sleeve T-Sh...,24.50,True,0.00,LRG,DGY,Dark Gray,https://www.duluthtrading.com/mens-longtail-t-...


In [7]:
# Filter to find the specific product/ sale price you want
sale_filter = df['SalePercent'] > 0
instock_filter = df['Orderable'] == True
size_filter = df['Size'] == 'MED'
# color_filter = df['Color'] == 'BHR'

df.loc[sale_filter & instock_filter & size_filter]

,Id,Name,Price,Orderable,SalePercent,Size,Color,ColorName,link
35,000524136,Men's Longtail T Relaxed Fit Short Sleeve T-Sh...,14.99,True,0.39,MED,AMR,Amber Brown,https://www.duluthtrading.com/mens-longtail-t-...


In [8]:
# Optional: Email the list to your email address